## **1. Setup**


---



In [ ]:
        def reset_target(scope):
            if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
                scope.io.pdic = 'low'
                time.sleep(0.1)
                scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
                time.sleep(0.1) #xmega needs more startup time
            elif "neorv32" in PLATFORM.lower():
                raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
            elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
                scope.io.nrst = 'low'
                time.sleep(0.25)
                scope.io.nrst = 'high_z'
                time.sleep(0.25)
            else:  
                scope.io.nrst = 'low'
                time.sleep(0.05)
                scope.io.nrst = 'high_z'
                time.sleep(0.05)

In [ ]:
import chipwhisperer as cw
import time
class CW_Setuper:
    # Param. for chipwhisperer initialization
    hardware_dir = '../../../'                                                              # path to the imported /hardware/victims/firmware folder
    chipwhisperer_init : dict = {
        "SCOPETYPE" : 'OPENADC',                                                            # Scope type ...

        "MAKEDIR" : './makefile_dir_2/',                                                    # Makefile folder relative path
        "CRYPTO_TARGET" : './',                                                             # Crypto target folder
        "FIRMWAREPATH" : f'{hardware_dir}hardware/victims/firmware',                         # Firmware path, path to compiler, simpleserial of chipwhisperer

        "PLATFORM" : 'CWLITEARM',                                                           # Chipwhisperer device type/Platform
        "SS_VER" :'SS_VER_2_1',                                                             # SimpleSerial version, in this case version 2
        "TARGET" :'simpleserial-target',                                                    # Target name, in this case simpleserial-target
    }
    firmware_bash_cmd = f''

    def __init__(self, firmware_reinit = False, re_init_dict = None):
        if firmware_reinit:
            print("⚠️ Reinit, initiated!!!")
            if re_init_dict is not None:
                for key in re_init_dict.keys():
                    chipwhisperer_init[key] = re_init_dict[key]
            self._init_firmware_()
        self._init_chipwhisperer_()
        cw.program_target(self.scope, cw.programmers.STM32FProgrammer, f"{self.chipwhisperer_init['MAKEDIR']}simpleserial-target-CWLITEARM.hex")
        reset_target(self.scope)
        print("✔️ Setuper initiated.")
    
    def _init_chipwhisperer_(self):# - Running additional ipynb file, containing basic setup of chipwhisperer, conditional of SETUP_VARS
        SS_VER = self.chipwhisperer_init["SS_VER"]
        PLATFORM = self.chipwhisperer_init["PLATFORM"]
        try:
            if not scope.connectStatus:
                scope.con()
        except NameError:
            scope = cw.scope()
        
        try:
            if SS_VER == "SS_VER_2_1":
                target_type = cw.targets.SimpleSerial2
            elif SS_VER == "SS_VER_2_0":
                raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
            else:
                target_type = cw.targets.SimpleSerial
        except:
            SS_VER="SS_VER_1_1"
            target_type = cw.targets.SimpleSerial
        
        try:
            target = cw.target(scope, target_type)
        except:
            print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
            print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
            scope = cw.scope()
            target = cw.target(scope, target_type)
        print("INFO: Found ChipWhisperer😍")

        if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
            prog = cw.programmers.STM32FProgrammer
        elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            prog = cw.programmers.XMEGAProgrammer
        elif "neorv32" in PLATFORM.lower():
            prog = cw.programmers.NEORV32Programmer
        elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
            prog = cw.programmers.SAM4SProgrammer
        else:
            prog = None
        time.sleep(0.05)
        scope.default_setup()
        self.scope = scope
        self.target = target
        print("✔️ INFO: Scope and Target, are set😍")
    
    def _init_firmware_(self):
        import subprocess
        self.firmware_bash_cmd = '''cd %s | make CRYPTO_TARGET=%s PLATFORM=%s FIRMWAREPATH=%s SS_VER=%s TARGET=%s> output.txt 2>warnings_new.txt'''
        
        # Deactivated block of code, because it throws error, because path
            #print(self.firmware_bash_cmd  % tuple(str(self.chipwhisperer_init[k]) for k in ('MAKEDIR', 'CRYPTO_TARGET', 'PLATFORM', 'FIRMWAREPATH', 'SS_VER',  'TARGET')))
            #result = subprocess.run(self.firmware_bash_cmd
            #                        % tuple(str(self.chipwhisperer_init[k]) for k in ('MAKEDIR', 'CRYPTO_TARGET', 'PLATFORM', 'FIRMWAREPATH', 'SS_VER',  'TARGET')),
            #                        shell=True,
            #                        executable='bin/bash')
        
        subprocess.call(f'firmware_setup.sh %s %s %s %s %s %s'
                        % tuple(str(self.chipwhisperer_init[k]) for k in ('MAKEDIR', 'CRYPTO_TARGET', 'PLATFORM', 'FIRMWAREPATH', 'SS_VER',  'TARGET')),
                        shell=True)
        print("✔️ Firmware maked.")
    def _scope_disc_(self):
        try:
            self.scope.dis()
            self.target.dis()
            print("✔️ Firmware disconnected.")
        except Exception as e:
            print("❌ Disconection failed because: ",e)